In [6]:
"""
This code takes in a csv file from the CTA bus website, removes any
incomplete bus data, and reformats the file to add columns for 
Weekday, Saturday, and Sunday - Holiday and saves the new
dataframe to a csv file.

Optional: column names as input?
"""
import pandas as pd

def bus_cleaning(csv_file, col_names):
    bus_data = pd.DataFrame()
    bus = pd.read_csv(csv_file, names = col_names)

    #determine how many bus lines to remove
    value_counts_BUS = bus['BUS'].value_counts()
    full_values = value_counts_BUS.max() #the is the number of counts for full data

    #keep track of which buses have full data or not
    buses_removed = []
    buses_kept = []
    for index, value in value_counts_BUS.items():
        if value < full_values:
            buses_removed.append(index)
        else:
            buses_kept.append(index)

    #drop the rows for any bus lines that do not have full data
    bus_cleaned = bus[~bus['BUS'].isin(buses_removed)]

    #ensure the AVERAGE data is an integeger
    bus['AVERAGE'] = pd.to_numeric(bus['AVERAGE'], errors='coerce')

    # to add columns for the Weekday, Saturday, Sunday-Holiday data, we need to
    # pivot the dataframe
    bus_pivoted = bus.pivot_table(
    index = ['BUS','YEAR','MONTH'],
    columns = 'DAY_TYPE',
    values = 'AVERAGE',
    aggfunc='mean' #not sure if this part is needed
    ).reset_index()

    #create a new column for weekly average (i.e. sum of the days of the week)
    bus_pivoted['Sum'] = bus_pivoted[['Weekday', 'Saturday', 'Sunday - Holiday']].sum(axis=1)
    bus_pivoted['Sum'] = bus_pivoted['Sum'].round().astype('Int64')

    #ensure the bus routes are strings
    bus_pivoted['BUS'] = bus_pivoted['BUS'].astype('string')
    bus_pivoted = bus_pivoted[bus_pivoted['BUS'] != '0']
    #drop the '0' bus
    

    bus_pivoted = bus_pivoted[['BUS', 'YEAR', 'MONTH', 'Weekday', 'Saturday', 'Sunday - Holiday', 'Sum']]

    #save new file
    new_file_name = csv_file[:-4]+"_cleaned.csv"
    bus_pivoted.to_csv(new_file_name, index = False)


In [7]:
bus_cleaning("CTA_Average_Bus_Ridership_1999_2024.csv",['BUS', 'YEAR','MONTH','DAY_TYPE','AVERAGE'])
